# Child Protection Plan (CPP) - Demographic Analysis

# Import Libraries

In [ ]:
#import bigquery
from google.cloud import bigquery
from google.cloud import bigquery_storage
 
#other needed libraries
import os
import pandas as pd
import numpy as np
import pandas_gbq
import seaborn as sns
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

import warnings
warnings.filterwarnings('ignore')
plt.style.use('tableau-colorblind10')
color='#702A7D'
grey_color = '#A9A9A9'

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jupyter/.config/gcloud/application_default_credentials.json"

#Instatiate BigQuery Client
client = bigquery.Client()

# Define Query for Import

In [ ]:
# CPP Data
cpp_query = """
SELECT
  a.person_id, a.YearOfBirth
  ,a.PCArea, a.EthnicOrigin, a.CPP_Category
  ,a.StartDate, a.EndDate
  ,DATE(p.birth_datetime) AS DateOfBirth
  ,p.gender_source_value AS Gender
FROM
    yhcr-prd-bradfor-bia-core.CB_2649.cb_bmbc_ChildrensSocialServices_CPP AS a
LEFT JOIN
    yhcr-prd-bradfor-bia-core.CB_2649.person AS p
ON
    a.person_id = p.person_id
"""

# Load Queries into dataframes

In [ ]:
cpp = pandas_gbq.read_gbq(cpp_query)

## CPP - Child Protection Plan

In [ ]:
# First few rows of cpp
cpp.head()

In [ ]:
# CPP columns overview
cpp.info()

In [ ]:
# Total null values per column
cpp.isnull().sum()

In [ ]:
# Locate entries that have missing DateOfBirth
cpp.loc[cpp.DateOfBirth.isnull()]

In [ ]:
# Fill the missing date of birth as 15th of January. All DoB have day of birth as 15th
cpp.loc[1643, 'DateOfBirth'] = '2008-01-15'
cpp.loc[2665, 'DateOfBirth'] = '2003-01-15'

In [ ]:
# Locate entries that have missing DateOfBirth
cpp.loc[cpp.DateOfBirth.isnull()]

In [ ]:
# Unique values in the gender column
cpp.Gender.unique()

In [ ]:
# Create a mapping dictionary to clean the gender column
gender_mapping = {
    'F': 'Female',
    'M': 'Male',
    '1': None, # Not sure if it's M or F, so we decide on None since it's not much
    '2': None, # Not sure if it's M or F, so we decide on None since it's not much
    'Male': 'Male',
    'Female': 'Female',
    'U': None,
    'N': None,  
    'null': None,
    None: None
}

# clean the gender column

cpp['Gender'] = cpp['Gender'].map(gender_mapping)

In [ ]:
# Convert the dates to datetime format
cpp['StartDate'] = pd.to_datetime(cpp['StartDate'])
cpp['EndDate'] = pd.to_datetime(cpp['EndDate'])
cpp['DateOfBirth'] = pd.to_datetime(cpp['DateOfBirth'])

# Verify
cpp.dtypes

In [ ]:
# Timeline of the dataset
minimum_start_date = np.min(cpp['StartDate'])
last_start_date = np.max(cpp['StartDate'])

print(f"The CPP Data starts from: {minimum_start_date}")
print(f"The last start date is: {last_start_date}")

In [ ]:
# Check for duplicates
cpp.duplicated().sum()

In [ ]:
# Locate the duplicate entry
cpp.loc[cpp.duplicated(keep=False)]

In [ ]:
# Drop the duplicate entry
cpp.drop_duplicates(inplace=True)

In [ ]:
# Number of Persons in the CPP
print(f'Number of Unique IDs: {cpp.person_id.nunique()}')

In [ ]:
# check for duplicate person_id to see if there are re-entries
print(f"Number of duplicate person_ids: {cpp.duplicated('person_id').sum()} \n")

duplicate_person_ids = cpp.loc[(cpp.duplicated('person_id', keep=False))].sort_values(by='person_id')
duplicate_person_ids

In [ ]:
# Display top 10 Persons with re-entries
duplicate_person_ids.person_id.value_counts().head(10)

## Data Exploration

In [ ]:
# Gender Distribution
plt.figure(figsize=(5,4))
ax = sns.countplot(data=cpp, 
                   x='Gender',
                   color=color)
ax.set_title(' CPP Gender Distribution')
ax.set_xlabel('Gender')

# Add counts to the bar
for container in ax.containers:
    ax.bar_label(container, fmt='%d')

plt.tight_layout()
plt.show()

In [ ]:
# Ethnicity Distribution
plt.figure(figsize=(8, 10))

# Calculate percentages
ethnicity_counts = cpp['EthnicOrigin'].value_counts(normalize=True) * 100
ethnicity_order = ethnicity_counts.sort_values(ascending=False).index[:12]

# Filter to top 12 ethnicities
top_ethnicity_counts = ethnicity_counts[ethnicity_order]

# Create the plot with percentages
ax = sns.barplot(
    x=top_ethnicity_counts.values,
    y=ethnicity_order,
    color=color
)
ax.set_title('Top 12 Ethnicity Distribution in CPP', fontsize=11)
ax.set_xlabel('Percentage (%)', fontsize=12)
ax.set_ylabel(None)
# Add percentage labels to the bars
for i, p in enumerate(ax.patches):
    width = p.get_width()
    ax.text(
        width + 0.3,
        p.get_y() + p.get_height()/2,
        f'{width:.1f}%',  
        ha='left',
        va='center',
        fontsize=10)
plt.tight_layout()
plt.savefig('../figs/CPP_Ethinicity_Dist.png', dpi=300)
plt.show()

In [ ]:
# PCArea Distribution
plt.figure(figsize=(16,6))
ax = sns.countplot(data=cpp, 
                   x='PCArea',
                   order= cpp['PCArea'].value_counts().sort_values(ascending=False).index,
                   color=color)
ax.set_title('PCArea Distribution')
ax.set_xlabel('Count')

# Add counts to the bar
for container in ax.containers:
    ax.bar_label(container, fmt='%d')

plt.tight_layout()
plt.show()

In [ ]:
# Statistics of the Area Distribution
print(f"Average number of children in CPP per area: {cpp.PCArea.value_counts().mean()}")
print(f"Median number of children in CPP per area: {cpp.PCArea.value_counts().median()}")
print(f"STD of children in CPP per area: {round(cpp.PCArea.value_counts().std(),2)}")

In [ ]:
"""
What's the most common reason for intervention?

"""

# CPP Category Distribution
plt.figure(figsize=(6,5))
ax = sns.countplot(data=cpp, 
                   x='CPP_Category',
                   order= cpp['CPP_Category'].value_counts().sort_values(ascending=False).index,
                   color=color)
ax.set_title('CPP Category Distribution')
ax.set_xlabel('Count')

# Add counts to the bar
for container in ax.containers:
    ax.bar_label(container, fmt='%d')

plt.tight_layout()
plt.show()

In [ ]:
"""
Is there pattern(s) in intervention reasons across areas?

"""


# Create a pivot table to count occurrences of each categories in each postcode area
pivot_table = cpp.pivot_table(index='PCArea', columns='CPP_Category', aggfunc='size', fill_value=0)

# Convert counts to percentages
percentage_table = pivot_table.div(pivot_table.sum(axis=1), axis=0) * 100

# Plot a heatmap
plt.figure(figsize=(14, 8))
sns.heatmap(percentage_table, annot=True, fmt=".1f", cmap="Purples", cbar_kws={'label': 'Percentage'})
plt.title('Distribution of CPP Categories by Postcode Area')
plt.xlabel('Category')
plt.ylabel('Postcode Area')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Create a pivot table to count occurrences of each reason in each postcode area
pivot_table = cpp.pivot_table(index='PCArea', columns='CPP_Category', aggfunc='size', fill_value=0)

# Convert counts to percentages by categories
percentage_table_by_category = pivot_table.div(pivot_table.sum(axis=0), axis=1) * 100

# Plot a heatmap
plt.figure(figsize=(14, 8))
sns.heatmap(percentage_table_by_category, annot=True, fmt=".1f", cmap="Purples", cbar_kws={'label': 'Percentage'})
plt.title('Category Percentage Distribution Across Postcode Areas')
plt.xlabel('Category')
plt.ylabel('Postcode Area')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
"""
What's the distribution of the categories by the Ethnicity and across Ethnicities?

"""
# Identify the top 10 most frequent ethnic origins
top_ethnic_origins = cpp['EthnicOrigin'].value_counts().nlargest(10).index

# Filter the DataFrame to include only the top 10 ethnic origins
filtered_cpp = cpp[cpp['EthnicOrigin'].isin(top_ethnic_origins)]

# Create a pivot table to count occurrences of each categories  in each Ethnic Origin
pivot_table_ethnicity = filtered_cpp.pivot_table(index='EthnicOrigin', columns='CPP_Category', aggfunc='size', fill_value=0)

# Convert counts to percentages by categories
percentage_table_by_ethnic_origin = pivot_table_ethnicity.div(pivot_table_ethnicity.sum(axis=1), axis=0) * 100

# Plot a heatmap
plt.figure(figsize=(14, 8))
sns.heatmap(percentage_table_by_ethnic_origin, annot=True, fmt=".1f", cmap="Purples", cbar_kws={'label': 'Percentage'})
plt.title('Distribution of Categories by Top 10 Ethnic Origins')
plt.ylabel('Ethnic Origin')
plt.xlabel('Category')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Identify the top 10 most frequent ethnic origins
top_ethnic_origins = cpp['EthnicOrigin'].value_counts().nlargest(10).index

# Filter the DataFrame to include only the top 10 ethnic origins
filtered_cpp = cpp[cpp['EthnicOrigin'].isin(top_ethnic_origins)]

# Create a pivot table to count occurrences of each categories  in each Ethnic Origin
pivot_table_ethnicity = filtered_cpp.pivot_table(index='EthnicOrigin', columns='CPP_Category', aggfunc='size', fill_value=0)

# Convert counts to percentages by categories
percentage_table_by_ethnic_origin = pivot_table_ethnicity.div(pivot_table_ethnicity.sum(axis=0), axis=1) * 100

# Plot a heatmap
plt.figure(figsize=(14, 8))
sns.heatmap(percentage_table_by_ethnic_origin, annot=True, fmt=".1f", cmap="Purples", cbar_kws={'label': 'Percentage'})
plt.title('Distribution of Categories Accross Top 10 Ethnic Origins')
plt.ylabel('Ethnic Origin')
plt.xlabel('Category')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


### What is the distribution of the Children in terms of Age at their time of entry into the intervention?

In [ ]:
"""
What is the distribution of the Children in terms of Age at their time of entry into the intervention?

"""

# Calculate the age of each children by substracting their birth year from the year they entered the CPP
cpp['AgeAtEntry'] = cpp.apply(
    lambda row: relativedelta(row['StartDate'], row['DateOfBirth']).years,
    axis=1
).astype('int')


max_age = cpp['AgeAtEntry'].max()

# Plot the distribution of the Ages
plt.figure(figsize=(8,6))
sns.histplot(data=cpp, x='AgeAtEntry', color=color, kde=True, bins=range(0, cpp['AgeAtEntry'].max() + 1));
plt.title(' Age Distribution of Children at the Time of Entry into CPP')
plt.ylabel('Frequency')
plt.xlabel('Age')

# Set x-ticks to be integers
plt.xticks(np.arange(0, max_age, step=1))
plt.tight_layout()
plt.show()

# Monthly Entry and Exit Trends in CPP

In [ ]:
# ENTRIES

# Count entry cases per month
monthly_entries = cpp['StartDate'].dt.to_period('M').value_counts().sort_index()

# Exclude the last month because the data didn't capture the whole month
last_month = monthly_entries.index[-1]
monthly_entries = monthly_entries[monthly_entries.index != last_month]

# Convert PeriodIndex to string for plotting
monthly_entries.index = monthly_entries.index.astype(str)

# EXITS

# Count exit cases per month
monthly_exits = cpp['EndDate'].dt.to_period('M').value_counts().sort_index()

# Exclude the last month because the data didn't capture the whole month
monthly_exits = monthly_exits[monthly_exits.index != last_month]
monthly_exits.index = monthly_exits.index.astype(str)

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(13, 10), height_ratios=[2, 1])

# Top plot: Entries and Exits trends
ax1.plot(monthly_entries.index, monthly_entries.values, marker='o', color=color, label='Monthly Entries')
ax1.plot(monthly_exits.index, monthly_exits.values, marker='o', color='red', label='Monthly Exits')
ax1.set_title(f'Monthly Trend of CPP Entries & Exits from {cpp.StartDate.min().date()} to {cpp.StartDate.max().date()} ')
ax1.set_xlabel('')
ax1.set_ylabel('Number of Cases')
ax1.legend()
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Bottom plot: Net Change
monthly_entries_aligned = monthly_entries.reindex(monthly_exits.index) # Ensures both series have same index (months).
difference = monthly_entries_aligned - monthly_exits

# Create bar plot for difference
bars = ax2.bar(difference.index, difference.values, color='green', alpha=0.6)

# Add horizontal line at y=0
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# Color bars based on positive/negative values
for bar, value in zip(bars, difference.values):
    if value < 0:
        bar.set_color('red')
    else:
        bar.set_color('green')

ax2.set_title('Net Change (Entries - Exits)')
ax2.set_xlabel('Month')
ax2.set_ylabel('Net Change')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

# Adjust layout
plt.tight_layout()
plt.savefig('../figs/EntriesVsExits.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# Print summary statistics
print("\nSummary of Net Changes:")
print(f"Average monthly net change: {difference.mean():.1f}")
print(f"Maximum increase: {difference.max():.0f}")
print(f"Maximum decrease: {difference.min():.0f}")
print(f"Months with net increase: {(difference > 0).sum()}")
print(f"Months with net decrease: {(difference < 0).sum()}")

## Entries into CPP for Each Financial Year

In [ ]:
# Define financial years
financial_years = [
    ('2017/04-2018/03', '2017-04-01', '2018-03-31'),
    ('2018/04-2019/03', '2018-04-01', '2019-03-31'),
    ('2019/04-2020/03', '2019-04-01', '2020-03-31'),
    ('2020/04-2021/03', '2020-04-01', '2021-03-31')
]

# Calculate entries for each financial year
entries_by_fy = []
for fy_label, start_date, end_date in financial_years:
    mask = (cpp['StartDate'] >= start_date) & (cpp['StartDate'] <= end_date)
    cpp_entries = cpp[mask].shape[0]
    entries_by_fy.append({
        'Financial Year': fy_label,
        'Total Entries': cpp_entries
    })

# Create a DataFrame for the results
result_df = pd.DataFrame(entries_by_fy)

# Display as a formatted table
print("\nTotal Entries by Financial Year:")
result_df

## What is the demand in intervention?

In [ ]:
# Align on the same monthly index; fill_value=0 ensures missing months get zero counts
monthly_entries_aligned = monthly_entries.reindex(monthly_exits.index, fill_value=0)

# Calculate monthly net change and the cumulative sum
difference = monthly_entries_aligned - monthly_exits
cumulative_in_intervention = difference.cumsum()  # Running total

cumulative_in_intervention.index = cumulative_in_intervention.index.astype(str)

# Plot the running total
plt.figure(figsize=(8, 5))

# Plot line without markers
plt.plot(cumulative_in_intervention.index, cumulative_in_intervention.values, 
         color=color, linewidth=2)


# Add markers only at start and end points
start_date = cumulative_in_intervention.index[0]
end_date = cumulative_in_intervention.index[-1]
start_value = cumulative_in_intervention.values[0]
end_value = cumulative_in_intervention.values[-1]

# Plot markers
plt.scatter([start_date, end_date], [start_value, end_value], 
            color=color, s=100, zorder=5)

# Annotate start and end values
plt.annotate(f'{int(start_value)}', 
            xy=(start_date, start_value),
            xytext=(-10, 10),  # Offset the text slightly
            textcoords='offset points',
            ha='right',
            va='bottom',
            color=color)

plt.annotate(f'{int(end_value)}', 
            xy=(end_date, end_value),
            xytext=(10, 10),  # Offset the text slightly
            textcoords='offset points',
            ha='left',
            va='bottom',
            color=color)

# Customize the plot
plt.title('Running Total of Individuals In CPP', pad=20)
plt.xlabel('Month')
plt.ylabel('Cumulative Count')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

# Show every 6th month label
all_xticks = plt.gca().get_xticks()
plt.gca().set_xticks(all_xticks[::6])
plt.xticks(rotation=45)
    
# Remove chart borders
sns.despine(left=True, right=True, top=True, bottom=True)
plt.tight_layout()
plt.savefig('../figs/cpp_cummsum_in_intervention.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# Number of days in intervention

In [ ]:
# Subtract start date from end date to get number of days
cpp['num_of_days_in_intervention'] = (cpp['EndDate'] - cpp['StartDate']).dt.days

#Convert to integer wile preserving NA values using Int64
cpp['num_of_days_in_intervention'] = cpp['num_of_days_in_intervention'].astype('Int64')

In [ ]:
# Statistics of CPP Intervention Duration
print("\nBasic Statistics of Intervention Duration:")
print(cpp['num_of_days_in_intervention'].describe().round(2))

print("\nSummary of missing values:")
print(f"Percentage missing: {(cpp['num_of_days_in_intervention'].isna().sum()/len(cpp))*100:.2f}%")

In [ ]:
# Distribution of Intervention Durations

# Define the color for the non-highlighted bins
grey_color = '#A9A9A9'  

# Calculate bin edges from 0 to max value with steps of 30 days
bin_edges = np.arange(0, max(cpp['num_of_days_in_intervention'].dropna()) + 30, 30)

# Create histogram with calculated number of bins
plt.figure(figsize=(10, 6))

# Plot the histogram
n, bins, patches = plt.hist(cpp['num_of_days_in_intervention'].dropna(), 
                            bins=bin_edges,
                            color=grey_color,  # Default color for all bars
                            edgecolor='black',
                            alpha=0.7)

# Highlight the 90-day bin
for i, (patch, bin_start) in enumerate(zip(patches, bins)):
    if bin_start == 60:  # Check if the bin starts at 60 days
        patch.set_facecolor(color)  # Set the color for the 60-90 days bin
        patch.set_edgecolor(color)
        patch.set_alpha(0.9)  # Set transparency
        # Annotate the 60-90 day bin
        plt.text(bin_start + 15, n[i] + 5, 'Between 60-90 days', 
                 ha='center', va='bottom', fontsize=12, color=color, weight='normal')

# Customize the plot
plt.title('Distribution of Intervention Durations (30 days bin)', fontsize=14)
plt.xlabel('Number of Days', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(True, alpha=0.3)

# Remove chart borders (spines)
sns.despine(left=True, right=True, top=True, bottom=True)

# Save and show the plot
plt.tight_layout()
plt.savefig('../figs/num_of_days.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

## Age group distribution across the intervention

In [ ]:
# Group Entry Ages
bins = [0, 1, 4, 9, 15, 16]
labels = ['Under 1', '1-4', '5-9', '10-15', '16+']

# Create new age group column
cpp['entry_agegroup'] = pd.cut(cpp['AgeAtEntry'], bins=bins, labels=labels, right=True)

# Calculate percentages
age_counts = cpp['entry_agegroup'].value_counts(normalize=True).sort_index() * 100

# Plot distribution of entry age group
plt.figure(figsize=(6, 4))
ax = sns.barplot(x=age_counts.index, y=age_counts.values, color=color)

# Add percentage labels on each bar
for i, p in enumerate(ax.patches):
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2.,
            height + 1,
            f'{height:.1f}%',
            ha="center", fontsize=10)

plt.title('Age Group Distribution in CPP', fontsize=14, pad=20)
plt.xlabel('Entry Age Group', fontsize=12)
plt.ylabel('Percentage', fontsize=12)
plt.ylim(0, max(age_counts.values) * 1.15)
plt.tight_layout()
plt.savefig(f'../figs/cpp_entryagegroup_dist.png', dpi=300)
plt.show()

In [ ]:
# Category distribution by Age Group

# Create a count DataFrame
count_df = cpp.groupby(['entry_agegroup', 'CPP_Category']).size().reset_index(name='count')

# Create the line plot
sns.lineplot(data=count_df, x='entry_agegroup', y='count', hue='CPP_Category', marker='o')
plt.title('Distribution of Categories Across Age Groups')
plt.tight_layout()
plt.show()